# PROJECT
## Company: Sparkify
## Goal: Data Modelling and ETL Pipeline for Processing and Analysis of Song's data

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

print(f"Amount of Files in `file_path_list`: {len(file_path_list)}")

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it      
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [ ]:
# Create Cluster
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [ ]:
# Create Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify_key
WITH REPLICATION =
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

In [ ]:
# Set KEYSPACE
session.set_keyspace('sparkify_key')

In [ ]:
# Drop table is exists
try:
    session.execute("DROP TABLE IF EXISTS artist_song_lenght_tb ;")
except Exception as e:
    print("Error DROPPING TABLE(s)!")
    print(e)

## Create table for retrieval of 'artist', 'song title' and 'song's length' was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_song_lenght_tb \
            (sessionId VARCHAR, itemInSession VARCHAR, artist VARCHAR, song VARCHAR, lenght VARCHAR, \
                PRIMARY KEY (sessionId, itemInSession))"
                ## PRIMARY KEY determined based on above requested values

#Execute above CREATE statement
try:
    session.execute(query)
except Exception as e:
    print('Error CREATING TABLE artist_song_lenght_tb')
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## INSERT data into the table
        query = "INSERT INTO artist_song_lenght_tb \
                    (sessionId, itemInSession, artist, song, lenght)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[8], line[3], line[0], line[9], line[5]))

In [ ]:
## REQUESTED VALUES = sessionId = 338, and itemInSession = 4

# Asks and assign user input to object
s_id = str(input('Enter `sessionId` value: '))
item_is = str(input('Enter `itemInSession` value: '))

# Assign SELECT statement to object
query = f"SELECT artist, song, lenght FROM artist_song_lenght_tb \
            WHERE sessionId = '{s_id}' AND itemInSession = '{item_is}'"

# Execute above query
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query!")
    print(e)

# Start BOLD and End BOLD
sb = '\033[1m'
eb = '\033[0m'

#Checks if 'rows' is not empty, otherwise, prints message
if rows.current_rows != []:
    for row in rows:
        print(f"\n{sb}Artist:{eb} {row.artist} \n{sb}Song:{eb} {row.song} \n{sb}Lenght:{eb} {row.lenght}")
else:
    print(f"{sb}No value(s) found for this search!{eb}")

In [ ]:
#Drop Table is Exists
try:
    session.execute("DROP TABLE IF EXISTS artist_song_user_sorted_tb ;")
except Exception as e:
    print("Error DROPPING TABLE(s)!")
    print(e)

## Create table for retrieval of 'name of artist', 'song' (sorted by itemInSession) and 'user' (first and last name)
## Requested values -> for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_user_sorted_tb \
            (userId VARCHAR, sessionId VARCHAR, artist VARCHAR, song VARCHAR, \
                firstName VARCHAR, lastName VARCHAR, itemInSession VARCHAR,\
                    PRIMARY KEY ((userId, sessionId), itemInSession))"
                    ## PRIMARY KEY / CLUSTERING determined based on above requested values and sorting

#Execute above CREATE statement
try:
    session.execute(query)
except Exception as e:
    print('Error CREATING TABLE artist_song_user_sorted_tb')
    print(e)                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## INSERT data into the table
        query = "INSERT INTO artist_song_user_sorted_tb \
                    (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[10], line[8], line[0], line[9], line[1], line[4], line[3]))

In [ ]:
## RETRIEVES VALUES BY `userId` and `sessionId`
## Requested values -> for userid = 10, sessionid = 182

# Start BOLD and End BOLD
sb = '\033[1m'
eb = '\033[0m'

print(f"{sb}RETRIEVE VALUES BY `userId` and `sessionId`{eb}\n")

#User input for `userId` and `sessionId`
user_id = str(input(f"{sb}Enter `userId` value: {eb}"))
session_id = str(input(f"{sb}Enter `sessionId` value: {eb}"))

# Assign SELECT statement to object
query = f"SELECT artist, song, firstName, lastName FROM artist_song_user_sorted_tb \
            WHERE userId = '{user_id}' AND sessionId = '{session_id}'"

# Execute above SELECT statement
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query!")
    print(e)

#Checks if 'rows' is not empty, otherwise, prints message
if rows.current_rows != []:
    print(f"\n{sb}Artist {'Song': >{17}} {'User Name': >{58}}{eb}")
    for row in rows:
        print(f"{row.artist: <{18}}  {row.song: <{53}} {row.firstname} {row.lastname}")    
else:
    print(f"\n{sb}No value(s) found for this search!{eb}")

In [ ]:
#Drop Table is Exists
try:
    session.execute("DROP TABLE IF EXISTS user_name_spec_music_tb ;")
except Exception as e:
    print("Error DROPPING TABLE!")
    print(e)

## Create table for retrieval of 'user name's (first and last) who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_name_spec_music_tb \
            (song VARCHAR, userId VARCHAR, firstName VARCHAR, lastName VARCHAR, \
                    PRIMARY KEY (song, userId))"
                    ## PRIMARY KEY determined so to uniquely identify requested data

# Execute above CREATE statement
try:
    session.execute(query)
except Exception as e:
    print('Error CREATING TABLE user_name_spec_music_tb')
    print(e)                 

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## INSERT data into the table
        query = "INSERT INTO user_name_spec_music_tb \
                    (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], line[10], line[1], line[4]))

In [ ]:
## RETRIEVES `firstName` AND `lastName` BY `song name`
## Requested value -> song = 'All Hands Against His Own'

# Start BOLD and End BOLD
sb = '\033[1m'
eb = '\033[0m'

print(f"{sb}RETRIEVE VALUES BY `song`{eb}\n")

#User input for `song` name
song_name = str(input(f"{sb}Enter `song` name: {eb}"))

# Assign SELECT statement to object
query = f"SELECT firstName, lastName FROM user_name_spec_music_tb \
            WHERE song = '{song_name}'"

# Execute above SELECT statement
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query!")
    print(e)


#Checks if 'rows' is not empty, otherwise, prints error message
if rows.current_rows != []:
    print(f"\n{sb}First Name {'Last Name': >{17}} {eb}")
    for row in rows:
        print(f"{row.firstname: <{18}}  {row.lastname: <{53}}")    
else:
    print(f"\n{sb}No value(s) found for this search!{eb}")

In [ ]:
## Drop all tables before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS artist_song_lenght_tb")
    session.execute("DROP TABLE IF EXISTS artist_song_user_sorted_tb")
    session.execute("DROP TABLE IF EXISTS user_name_spec_music_tb")
except Exception as e:
    print("Error DROPPING tables!")
    print(e)

In [ ]:
## Close the session and cluster connection¶
session.shutdown()
cluster.shutdown()